In [3]:
import random
import numpy as np 
from functools import reduce 



In [4]:
###---------------------------------------------------------------------###
###-------------------------------------> Helper functions

def f(a : str) -> str:
    return a+"o"
def g(a : str) -> str:
    return a+"K"
def h(a : str) -> str:
    return a+a
def ag1(a : str, b : str) -> str:
    return a+"-"+b
def ag2(a : str, b : str) -> str:
    return b+"-"+a



 

In [5]:
from typing import Callable
import random

rootType = Callable[[str,str], float]
nodeType = Callable[[str], str]
leafType = str

class SimTree():

    def __init__(self, value : leafType | nodeType | rootType = None, child : list = None):
        self.value = value 
        self.child = child or []
        self.isRoot = lambda x: True if(len(x.child) == 2) else False
        self.isNode = lambda x: True if(len(x.child) == 1) else False
        self.isLeaf = lambda x: True if(len(x.child) == 0) else False
        
    def __str__(self):
        if self.isRoot(self): #root 
            tree_list = str([self.value.__name__,
                         self.child[0].__str__(),
                         self.child[1].__str__()])
        elif self.isNode(self): #nodes
            tree_list = [self.value.__name__,self.child[0].__str__()]
        else: #leaf
            tree_list = self.value
        return tree_list  
          
    def return_tree_asList(self):
        if self.isRoot(self): #root 
            tree_list = [self.value,
                         self.child[0].return_tree_asList(),
                         self.child[1].return_tree_asList()]
        elif self.isNode(self): #nodes
            tree_list = [self.value,self.child[0].return_tree_asList()]
        else: #leaf
            tree_list = self.value 
        return tree_list
      
    def compute(self):
        if self.isRoot(self): #root
            return self.value(self.child[0].compute(),
                              self.child[1].compute())
        elif self.isNode(self): #nodes
            return self.value(self.child[0].compute())
        else: #leaf
            return self.value
    
    def mutate(self, get_rd_function, probability : float):
        if not self.isLeaf(self):
            if random.random() < probability:
                self.value = get_rd_function(len(self.child),self.value)
            else:
                for c in self.child:
                    c.mutate(get_rd_function,probability)

    def set_leaf_value(self,value):
        if self.isNode(self):
            self.child[0].set_leaf_value(value)
        if self.isLeaf(self):
            self.value = value

    def set_leafs_value(self,values):
        x,y = values
        self.child[0].set_leaf_value(x)
        self.child[1].set_leaf_value(y)

    def get_Similarity_function(self):
        if self.isRoot(self): #root
            return self.value
        
    def get_transformations_functions(self):
        if self.isRoot(self) == 2: #root
            return flatten([self.child[0].get_transformations_functions,
                            self.child[1].get_transformations_functions])

        elif self.isNode(self) == 1: #nodes
            return self.value

    def find_depth(self,x):
        x+=1
        if self.isRoot(self):
            return max(self.child[0].find_depth(x), self.child[1].find_depth(x))
        if self.isNode(self):
            return self.child[0].find_depth(x)
        if self.isLeaf(self):
            return x    

    def get_depth(self):
        return self.find_depth(0)    


def tree_from_list(tree_list : list) -> SimTree:
    if len(tree_list) == 3: #root
        value = tree_list[0]
        left_child = tree_list[1]
        right_child = tree_list[2]
        return SimTree(value,
                [tree_from_list(left_child),
                tree_from_list(right_child)])
    elif len(tree_list) == 2: #nodes        
        value = tree_list[0]
        child = tree_list[1]
        return SimTree(value,[tree_from_list(child)])
    else: #leaf
        value = tree_list[0]
        return SimTree(value,[])

In [6]:
from similarity import *
from transformation import *

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Combal\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Combal\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Combal\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
TRANSFORMATION_FUNCTIONS = transformation_functions()           
SIMILARITY_FUNCTIONS = similarity_functions()
           
flatten = lambda l : [item for sublist in l for item in sublist]
def get_rd_function(nb_child,value):
    if nb_child == 2:
        function_list = SIMILARITY_FUNCTIONS
    else:
        function_list = TRANSFORMATION_FUNCTIONS
    flag = True
    while flag:
        new_value = random.choice(function_list)
        if new_value != value:
            flag = False
    return new_value

In [8]:
from datetime import datetime

dates = [
    '2006-10-28', '2005-11-15', '1936-11-07', '1955-09-08', '1936-12-07',
    '1937-04-12', '1974-12-02', '1972-06-30', '1955-07-06', '2006-09-09',
    '2007-11-06', '2007-11-13', '2009-09-29', '2014-03-25', '2001-12-04',
    '2004-09-06', '2004-11-10', '2004-11-23', '2004-12-09', '2005-09-27',
    '2011-09-27', '2000-09-12', '1999-09-14', '2003-11-11', '2004-02-05',
    '2006-04-04', '2005-11-16', '2005-11-24', '2008-09-23', '2010-10-19',
    '2005-09-13', '2005-09-14', '2006-03-07', '2009-09-08', '2011-11-01',
    '2003-06-03', '1969-02-10', '2017-04-28', '2022-10-24', '2004-05-10',
    '2018-03-14', '2002-04-11', '2013-03-09'
]

american_dates = []
european_dates = []

for date_str in dates:
    date_obj = datetime.strptime(date_str, '%Y-%m-%d')
    american_date = date_obj.strftime('%m/%d/%Y')
    american_dates.append(american_date)
    european_date = date_obj.strftime('%d/%m/%Y')
    european_dates.append(european_date)

# Print the converted dates
values = []
for a,e in zip(american_dates,european_dates):
    #print(a+" - "+e)
    values.append((a,e))


In [9]:
param_algo = {
    "population_size" : 100,
    "nb_generation"   : 20,
    "proba_mutation"  : 0.3,
    "proba_crossover" : 0.2,
    "proba_random"    : 0.1
    #"proba_ellitism"  : 0.5,
}

param_data = {
    "tree_max_depth" : 8,
    "similarity_functions" : similarity_functions(),
    "transformation_functions" : transformation_functions(),
    "values" : values
}



In [8]:
class SimGen():
    def __init__(self, param_algo : dict , param_data : dict):
        #main param
        self.population_size : int = param_algo["population_size"]
        self.nb_generation   : int = param_algo["nb_generation"]
        
        #evolution param
        self.proba_mutation  : float = param_algo["proba_mutation"]
        self.proba_crossover : float = param_algo["proba_crossover"]
        self.proba_random_tree : float = param_algo["proba_random"]
        self.mutation_population_size    : int = round(self.proba_mutation*self.population_size)
        self.crossover_population_size   : int = round(self.proba_crossover*self.population_size)
        self.random_tree_population_size : int = round(self.proba_random_tree*self.population_size)
        self.elitism_population_size     : int = self.population - (
            self.mutation_population_size+
            self.crossover_population_size+
            self.random_tree_population_size
        )
        #self.proba_ellitism  : float = param_algo["proba_ellitism"] #proba to keep best elem

        #candidate param
        self.tree_max_depth : int = param_data["tree_max_depth"]

        #init param
        self.population : list[SimTree] = [] #contain every candidate solution
        self.population_scores : list[int] = [0]*len(self.population) #every candidate has a score computed at the end of an generation
        self.similarity_functions : list[rootType] = param_data["similarity_functions"] 
        self.transformation_functions : list[nodeType] = param_data["transformation_functions"]

        #values param
        self.values : list[tuple[str,str]] = param_data["values"] #[(x1,y1),(x2,y2)...,(xn,yn)] such as (ei,p,xi) <=> (e'i,p',yi)
        

    def generate_random_tree(self):
        nb_transformation = random.randint(1,self.tree_max_depth)
        tf_left = np.random.choice(self.transformation_functions,nb_transformation,replace=False)
        tf_left = np.append(tf_left,["a"]) #TODO check comment mieux gerer le lift
        tf_right = np.random.choice(self.transformation_functions,nb_transformation,replace=False)
        tf_right = np.append(tf_right,["a"])
        sf = np.random.choice(self.similarity_functions,1)[0]
        def nest_list(lst):
            #format our list of function : [f,g,h,...]
            #into : [f,[g,[h,[...]]]]
            if len(lst) == 1:
                return [lst[0]]
            else:
                return [lst[0], nest_list(lst[1:])]

        tree_list = [sf,nest_list(tf_left),nest_list(tf_right)]
        tree = tree_from_list(tree_list)
        return tree

    def init_population(self):
        """
            Generate a population of size n with random tree
        
        """
        #check variance and stuff meta data TODO
        for _ in range(self.population_size):
            self.population.append(self.generate_random_tree())
    
            
    def fitness_function(self):
        '''
            Compute the score of a tree on every value pair
            Keep the mean as a score
        '''    
         
        for index,candidate in enumerate(self.population):
            for values in self.values:
                #compute score for each value pair for one tree
                candidate.set_leafs_value(values)
                score = candidate.compute()
                self.population_scores[index] += score
            self.population_scores[index] /= len(self.values)
            
    def evolve_population(self):
        #compute score for each tree
        self.fitness_function()

        #helper functions
        def remove_elem(pop : list,score_pop : list,index_list : list[int]) -> None:
            for i in index_list:
                pop.remove(i)
                score_pop.remove(i)
            
        #select best individuals
        score_index = list(np.argsort(self.population_scores))
        score_index.reverse()
        #elitism population
        best_individuals = [self.population[index] for index in score_index[0:self.elitism_population_size]]
        #remove best elem from population
        remove_elem(self.population,self.population_scores,score_index[0:self.crossover_population_size])
        #mutation population

    #TODO add or remove for mutation
    #TODO crossover

In [9]:
#TODO check how to implement value because don't need value when instance simtree

In [41]:
foo = [1,81,0,78,21,85,31,4,513,8,1]
a = list(np.argsort(foo))
a.reverse()
nb = 5
a[0:5]
bb = [foo[i] for i in a[0:5]]
bb
foo.remove(0)
foo

[1, 81, 78, 21, 85, 31, 4, 513, 8, 1]

In [10]:
sim = SimGen(param_algo,param_data)
sim.init_population()
sim.fitness_function()

In [11]:
print_pop = list(map(lambda x: str(x),sim.population))

In [10]:
a = [ag1,[f,[g,"a"]],[g,[g,[g,["b"]]]]]
tree = tree_from_list(a)
print(tree.compute())
print(tree)
tree.mutate(get_rd_function,0.4)
print(tree.compute())
print(tree)
print(tree.get_depth())

aKo-bKKK
['ag1', ['f', ['g', 'a']], ['g', ['g', ['g', 'b']]]]
0.5277777777777778
['jarowinkler_similarity', ['f', ['g', 'a']], ['g', ['g', ['g', 'b']]]]
5


In [14]:
a = [ag1,[f,[g,[""]]],[g,[g,[""]]]]
tree = tree_from_list(a)
print(tree)
tree.set_leafs_value(["b","c"])
list = tree.return_tree_asList()
np.random.choice(1,list)

['ag1', ['f', ['g', '']], ['g', ['g', '']]]


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (3,) + inhomogeneous part.

In [37]:
a = ["1",["2",["3",["4"]]]]
SimTree(v,c)
SimTree(v,SimTree(d,c))

NameError: name 'v' is not defined

In [48]:
a = [ag1,[f,[g,[""]]],[g,[g,[""]]]]
tree = tree_from_list(a)
tree.child[0].value
nb_nodes = tree.get_depth() 
#tree.child[eval(choice)].value
np.random.choice(tree.child).value


<function __main__.g(a: str) -> str>

In [31]:
a = [ag1,[f,[g,[""]]],[g,[g,[""]]]]
action = ["add","modify","remove"]

match np.random.choice(action):
    case "add":
        choice = np.random.choice(["left","right"])
        
        pass
    case "modify":
        pass
    case "remove":
        pass

'remove'

In [14]:
k = [(1,2),(4,5),(3,8),(8,9)]
compute_score = lambda elem:(x:=[elem[0],y:=[elem[1]]], x[0]+y[0])[-1]
list(map(compute_score,k))



[3, 9, 11, 17]